# Play bot tournament

---

Author: S. Menary [sbmenary@gmail.com]

Date  : 2023-01-18, last edit 2023-01-23

Brief : Play a tournment between two bots to determine which is stronger

---

## Imports

---

In [1]:
##=====================================##
##  All imports should be placed here  ##
##=====================================##

##  Python core libs
import pickle, sys, time

##  PyPI libs
import numpy as np

##  Local packages
from connect4.utils    import DebugLevel
from connect4.game     import BinaryPlayer, GameBoard, GameResult
from connect4.MCTS     import PolicyStrategy
from connect4.bot      import Bot_NeuralMCTS, Bot_VanillaMCTS
from connect4.parallel import generate_from_processes
from connect4.methods  import play_bot_game


In [2]:
##=====================================##
##  Print version for reproducibility  ##
##=====================================##

print(f"Python version is {sys.version}")
print(f"Numpy  version is {np.__version__}")


Python version is 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:25:29) [Clang 14.0.6 ]
Numpy  version is 1.23.2


---

##  Play tournament or load past results

---

In [3]:
##=================##
##  Global config  ##
##=================##

load_from_file = False

idx_bot1 = 0
idx_bot2 = 7

bot1_policy = PolicyStrategy.GREEDY_POSTERIOR_POLICY
bot2_policy = PolicyStrategy.GREEDY_POSTERIOR_POLICY

if idx_bot1 == idx_bot2 :
    bot1_policy, bot2_policy = sorted([bot1_policy, bot2_policy])

num_proc           = 7
num_games_per_proc = 50
base_seed          = int(time.time())
mon_freq           = 3
duration           = .2
discount           = .99
num_data           = num_proc*num_games_per_proc

model1_name   = f"v{idx_bot1}: {bot1_policy.name}"
model2_name   = f"v{idx_bot2}: {bot2_policy.name}"
model1_fname  = f"../models/.neural_model_v{idx_bot1}.h5" if idx_bot1 else ""
model2_fname  = f"../models/.neural_model_v{idx_bot2}.h5" if idx_bot2 else ""
results_fname = f"../data/" + f"bot_tournament_v{idx_bot1}_{bot1_policy.name}_vs_v{idx_bot2}_{bot2_policy.name}_duration{duration:.2f}_n{num_data}".replace(".","p") +".pickle"

print(f"Model1: {model1_name} at file '{model1_fname}'")
print(f"Model2: {model2_name} at file '{model2_fname}'")
print(f"Results file: {results_fname}")


Model1: v0: GREEDY_POSTERIOR_POLICY at file ''
Model2: v7: GREEDY_POSTERIOR_POLICY at file '../models/.neural_model_v7.h5'
Results file: ../data/bot_tournament_v0_GREEDY_POSTERIOR_POLICY_vs_v7_GREEDY_POSTERIOR_POLICY_duration1p00_n350.pickle


In [4]:
##======================================================================##
##  Define method allowing games to be played in parallel subprocesses  ##
##======================================================================##

##  N.B. connect4.neural import must be performed inside the method executed inside each child process
##  to avoid a deadlock caused when a tf session has already been created in __main__

def bot_tournament_process(proc_idx, num_games, out_queue, argv) :
    from connect4.neural import load_model
    model1_fname, model2_fname, bot1_policy, bot2_policy, duration, discount, base_seed = argv
    model1 = load_model(model1_fname) if len(model1_fname) > 0 else None
    model2 = load_model(model2_fname) if len(model2_fname) > 0 else None
    np.random.seed(base_seed+proc_idx)
    for game_idx in range(num_games) :
        _ = play_bot_game(model1, model2, duration, discount, bot1_policy=bot1_policy, bot2_policy=bot2_policy)
        out_queue.put(_)


In [5]:
##=====================================================================##
##  Load old results, or run parallel subprocesses to obtain new ones  ##
##=====================================================================##

if load_from_file :
    print(f"Loading past results from file {results_fname}")
    loaded  = pickle.load(open(results_fname, "rb"))
    results = loaded["results"]
    for key, val in loaded.items() :
        if key == "results" : continue
        print(f"  Found saved config value {key} = {val}")
else :
    print(f"Generating {num_proc*num_games_per_proc} results with base seed {base_seed}")
    print(f"Using duration = {duration:.3f}\nUsing discount = {discount:.3f}")
    results = generate_from_processes(
        func      = bot_tournament_process,
        func_args = [model1_fname, model2_fname, bot1_policy, bot2_policy, duration, discount, base_seed],
        num_proc  = num_proc, 
        num_results_per_proc = num_games_per_proc, 
        mon_freq  = mon_freq)
 

Generating 350 results with base seed 1674550572
Using duration = 1.000
Using discount = 0.990
Generated 350 / 350 results [t=2078.41s]
Generation complete [t=2078.41s] [n=350]


In [6]:
##====================================##
##  Save results for later reference  ##
##====================================##

if load_from_file :
    print("Nothing to save because we loaded results from file")
else :
    print(f"Saving results to file {results_fname}")
    to_save = {"results"     : results,
               "num_results" : len(results),
               "base_seed"   : base_seed,
               "duration"    : duration,
               "discount"    : discount}
    pickle.dump(to_save, open(results_fname, "wb"))


Saving results to file ../data/bot_tournament_v0_GREEDY_POSTERIOR_POLICY_vs_v7_GREEDY_POSTERIOR_POLICY_duration1p00_n350.pickle


---

##  Report on the scores

---

In [7]:
##========================##
##  Report on the scores  ##
##========================##

score = [[0, 0, 0, 0], [0, 0, 0, 0]]

for result in results :
    score_idx = 0 if result["model1"] == BinaryPlayer.X else 1
    result_idx = 0
    if result["result"] == GameResult.X    : result_idx = 1
    if result["result"] == GameResult.DRAW : result_idx = 2
    if result["result"] == GameResult.O    : result_idx = 3
    score[score_idx][result_idx] += 1
    
print("Results (LHS player goes first):")
print(f"[{model1_name}]".rjust(45)+f"  {score[0][1]} - {score[0][2]} - {score[0][3]}  [{model2_name}]")
print(f"[{model2_name}]".rjust(45)+f"  {score[1][1]} - {score[1][2]} - {score[1][3]}  [{model1_name}]")
    

Results (LHS player goes first):
                [v0: GREEDY_POSTERIOR_POLICY]  66 - 12 - 96  [v7: GREEDY_POSTERIOR_POLICY]
                [v7: GREEDY_POSTERIOR_POLICY]  112 - 7 - 57  [v0: GREEDY_POSTERIOR_POLICY]
